In [1]:
from random import randrange
import tkinter as tk
import sys
import mysql.connector
import bcrypt
import hashlib

In [2]:
tab_level = {
    1: {
        'the_range': 10,
        'nbr_essaie': 3,
    },
    2: {
        'the_range': 20,
        'nbr_essaie': 5,
    },
    3: {
        'the_range': 30,
        'nbr_essaie': 7,
    }
}

pay_game = {
    1: 2,
    2: 1,
    3: 0.5,
    4: 0,
}


regles = [
    "Si vous devinez mon nombre dès le premier coup, vous gagnez le double de votre mise !",
    "Si vous le devinez au 2è coup, vous gagnez exactement votre mise !",
    "Si vous le devinez au 3è coup, vous gagnez la moitiè votre mise !",
    "Si vous ne le devinez pas au 3è coup, vous perdez votre mise et vous avez le droit : ",
    "- de retenter votre chance avec l'argent qu'il vous reste pour reconquérir le level perdu.",
    "- de quitter le jeu.",
    "Dès que vous devinez mon nombre : vous avez le droit de quitter le jeu et de partir avec vos gains OU de continuer le jeu en passant au level supérieur."
]

In [3]:
class Database:
        
    def __init__(self, myUser, myPassword, myHost, myDatabase):
        self.user = myUser
        self.password = myPassword
        self.host = myHost
        self.database = myDatabase
    
    def getConnector(self):
        cnx = mysql.connector.connect(user=self.user, password=self.password, host=self.host, database=self.database)
        return cnx

In [4]:
class Entity:
    def __init__(self): 
#         db = Database("casino04", "casino04", "db4free.net", "casino_bdd")
        db = Database("root", "", "localhost", "casino_bdd")
        self.db = db.getConnector()

    def getConnexion(self):
        return self.db

In [5]:
class UtilisateurEntity(Entity):
    
    def __init__(self):
        super().__init__()
        
    def getUserByPseudo(self, pseudo, password):
        connexion = self.getConnexion()
        mycursor = connexion.cursor(prepared=True)
        query = """ SELECT 
                        u.id, 
                        u.pseudo, 
                        u.solde, 
                        u.date_inscription, 
                        MAX(p.level) as niveau_max
                    FROM utilisateur u
                    JOIN partie p ON u.id = p.user_id
                    WHERE u.pseudo = %s 
                    AND u.password = %s """
        mycursor.execute(query, [pseudo, password])        
        row = mycursor.fetchone()
        if row[0] is not None:
            user = Utilisateur(row[0],row[1],row[2], row[3], row[4])
        else:
            user = None
        return user
    
    def getUserById(self, user_id):
        connexion = self.getConnexion()
        mycursor = connexion.cursor(prepared=True)
        query = """ SELECT 
                        u.id, 
                        u.pseudo, 
                        u.solde, 
                        u.date_inscription, 
                        MAX(p.level) as niveau_max
                    FROM utilisateur u
                    JOIN partie p ON u.id = p.user_id
                    WHERE u.id = %s """
        mycursor.execute(query, [user_id])        
        row = mycursor.fetchone()
        user = Utilisateur(row[0],row[1],row[2], row[3], row[4])
        return user
    
    def checkPseudo(self, pseudo):
        connexion = self.getConnexion()
        mycursor = connexion.cursor(prepared=True)
        query = """ SELECT id FROM utilisateur WHERE pseudo = %s """
        mycursor.execute(query, [pseudo])        
        row = mycursor.fetchone()
        if row is not None:
            return False
        else:
            return True
    
    def registerUser(self, pseudo, password):
        connexion = self.getConnexion()
        mycursor = connexion.cursor(prepared=True)
        query = """ INSERT INTO utilisateur (id, pseudo, password, solde, date_inscription ) VALUES (NULL, %s, %s, NULL, now())"""
        mycursor.execute(query, [pseudo, password])
        connexion.commit()
        user_id = mycursor.lastrowid
        return user_id
    
    def updateUser(self, userObj):
        solde = userObj.getSolde()
        user_id = userObj.getUser_id()
       
        connexion = self.getConnexion()
        mycursor = connexion.cursor(prepared=True)
        query = """ UPDATE utilisateur SET solde = %s WHERE id = %s """
        mycursor.execute(query, [solde, user_id])
        connexion.commit()
    

In [6]:
class PartieEntity(Entity):
    
    def __init__(self):
        super().__init__()
    
    def newPartie(self, objPartie):
        
        mise = objPartie.getMise()
        level = objPartie.getLevel()
        gain = objPartie.getGain()
        nb_coup = objPartie.getNb_coup()
        resultat = objPartie.getResultat()
        user_id = objPartie.getUser_id()
        
        connexion = self.getConnexion()
        mycursor = connexion.cursor(prepared=True)
        query = """ INSERT INTO partie 
                        (id, mise, level, gain, nb_coup, resultat, date_game, user_id) 
                    VALUES 
                        (NULL, %s, %s, %s, %s, %s, now(), %s)"""
        mycursor.execute(query, [mise, level, gain, nb_coup, resultat, user_id])
        connexion.commit()
    
    def getListPartieOfUser(self, user_id):
        
        connexion = self.getConnexion()
        mycursor = connexion.cursor(prepared=True)
        query = """ SELECT id, mise, level, gain, nb_coup, resultat, date_game, user_id 
                    FROM partie 
                    WHERE user_id = %s """
        mycursor.execute(query, [user_id])
        rows = mycursor.fetchall()
        tab_partie = list()
        for row in rows:
            row = Partie(row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7])
            tab_partie.append(row)
        return tab_partie
    
    def getClassementGame(self):
        
        connexion = self.getConnexion()
        mycursor = connexion.cursor(prepared=True)
        query = """ SELECT COUNT(partie.id) as nbr_win, utilisateur.pseudo
                    FROM partie 
                    JOIN utilisateur ON partie.user_id = utilisateur.id
                    WHERE partie.resultat = 1
                    GROUP BY partie.user_id 
                    LIMIT 10 """
        mycursor.execute(query)
        rows = mycursor.fetchall()
        tab_class = list()
        for row in rows:
            tab_class.append(row)
        return tab_class
        

In [7]:
class Utilisateur:

    def __init__(self, user_id, pseudo, solde, date_inscription, niveau_max):
        self.user_id = user_id
        self.pseudo = pseudo
        self.solde = solde
        self.date_inscription = date_inscription
        if(niveau_max):
            self.niveau_max = niveau_max
        else:
            self.niveau_max = 0
        
        
    def getUser_id(self):
        return self.user_id
        
    def getPseudo(self):
        return self.pseudo
    
    def getSolde(self):
        return self.solde
    
    def getDateInscription(self):
        return self.date_inscription
    
    def getNiveau_max(self):
        return self.niveau_max
    
    
    
    def setUser_id(self, user_id):
        self.user_id = user_id
    
    def setPseudo(self, pseudo):
        self.pseudo = pseudo
    
    def setSolde(self, solde):
        self.solde = solde
    
    def setDateInscription(self, date_inscription):
        self.date_inscription = date_inscription
        
    def setNiveau_max(self, niveau_max):
        self.niveau_max = niveau_max

In [8]:
class Partie:
     
    def __init__(self, partie_id, mise, level, gain, nb_coup, resultat, date_game, user_id):
        self.partie_id = partie_id
        self.mise = mise
        self.level = level
        self.gain = gain
        self.nb_coup = nb_coup
        self.resultat = resultat
        self.date_game = date_game
        self.user_id = user_id
        
    def getPartie_id(self):
        return self.partie_id
    
    def getMise(self):
        return self.mise
    
    def getLevel(self):
        return self.level
    
    def getGain(self):
        return self.gain
    
    def getNb_coup(self):
        return self.nb_coup

    def getResultat(self):
        return self.resultat

    def getDate_game(self):
        return self.date_game

    def getUser_id(self):
        return self.user_id
    
    
    
    def setPartie_id(self, level):
        self.partie_id = partie_id

    def setMise(self, mise):
        self.mise = mise

    def setLevel(self, level):
        self.level = level

    def setGain(self, gain):
        self.gain = gain

    def setNb_coup(self, nb_coup):
        self.nb_coup = nb_coup

    def setResultat(self, resultat):
        self.resultat = resultat

    def setDate_game(self, date_game):
        self.date_game = date_game

    def setUser_id(self, user_id):
        self.user_id = user_id

In [9]:
class UtilisateurController:
    
    def loginUser(self):
        while True :
            pseudo = input("\n\t- Quel est votre pseudo ? \n")
            password = input("\n\t- Quel est votre mot de passe ? \n")
            password_hash = self.hashPassword(password)
            userEnt = UtilisateurEntity();
            userObj = userEnt.getUserByPseudo(pseudo, password_hash)
            if userObj is not None:
                return userObj
            print("\n Votre pseudo ou mot de passe est incorrecte")
                
        
    def registerUser(self):
        userEnt = UtilisateurEntity();
        while True :
            pseudo = input("\t- Choisisez un pseudo ? \n")
            password = input("\t- Choisisez un mot de passe \n")
            if(userEnt.checkPseudo(pseudo)):
                break
            else:
                print("Ce pseudo n'est pas disponible")
        
        password_hash = self.hashPassword(password)
        userEnt = UtilisateurEntity();
        user_id = userEnt.registerUser(pseudo, password_hash)
        userObj = userEnt.getUserById(user_id)
        userObj.setSolde(10)
        return userObj
    
    def saveUser(self, userObj):
        userEnt = UtilisateurEntity();
        userEnt.updateUser(userObj)
        
    def hashPassword(self, password):
    
        hash_object = hashlib.sha256(str(password).encode('utf-8'))
        pass_hash = hash_object.hexdigest()
        return pass_hash
    
    def calculStats(self, user_id):
        entPartie = PartieEntity();
        listObjPartie = entPartie.getListPartieOfUser(user_id)
        nbrPartie = 0
        nbrWin = 0
        for objPartie in listObjPartie:
            nbrPartie += 1
            if(objPartie.getResultat() == 1):
                nbrWin += 1
        win_rate = (nbrWin * 100) / nbrPartie
        print("\n Vos statistiques sont :")
        print("\n\t Nombre partie : ",nbrPartie) 
        print("\t Nombre de victoire : ",nbrWin)
        print("\t Votre taux de victoire est de",round(win_rate),"%")
        

In [10]:
class PartieController:
    
    def __init__(self, userObj):
        self.userObj = userObj
        self.level = 1
    
    def playCore(self):
        print("\n Hello ",self.userObj.getPseudo(),", vous avez ",self.userObj.getSolde()," €, Très bien ! Installez vous SVP à la table de pari.\n")
        self.wichMode()
    
    def wichMode(self):
        print("\n Que souhaitez vous faire ? (a/b/c/d)\n")
        print("\t (a) : Jouer au casino")
        print("\t (b) : Explication des règles")
        print("\t (c) : Voir mes statistiques")
        print("\t (d) : Classements des joueurs")
        print("\t (e) : Quitter")
        reponse = input()
        while True:
            if reponse.lower() == "a":
                self.modePlay()
                break
            elif reponse.lower() == "b":
                self.modeRegles()
                break
            elif reponse.lower() == "c":
                self.modeStats()
                break
            elif reponse.lower() == "d":
                self.modeClassement()
                break
            elif reponse.lower() == "e":
                self.modeQuitter()
                break
            else:
                print("\n Que souhaitez vous faire ? (a/b/c/d)\n")
                print("\t (a) : Jouer au casino")
                print("\t (b) : Explication des règles")
                print("\t (c) : Voir mes statistiques")
                print("\t (d) : Classements des joueurs")
                print("\t (e) : Quitter")
                reponse = input()
            
    def modePlay(self):
        while True:
            if(self.userObj.getNiveau_max() == 3):
                self.level = self.wichLevel()
                self.gameCasino()
            if(self.gameCasino() == True and self.level < 3):
                self.level += 1
            if(boolean_question("- Je ne comprends pas votre réponse. Souhaitez-vous continuer la partie (O/N) ?") == False):
                if (not self.userObj.getSolde()):
                    print("\t- Navré, vous avez tout perdu.. N'hésitez pas à revenir pour faire mieux la prochaine fois !")
                else:
                    print("\t- Merci d'avoir joué ! Vous repartez du casino avec {}€".format(self.userObj.getSolde()))
                userCtr = UtilisateurController()
                userCtr.saveUser(self.userObj)
                break
            self.wichMode()
                
    def modeRegles(self):
        print("""\n \t- Si vous devinez mon nombre dès le premier coup, vous gagnez le double de votre mise !\n
            \t- Si vous le devinez au 2è coup, vous gagnez exactement votre mise !\n
            \t- Si vous le devinez au 3è coup, vous gagnez la moitiè votre mise !\n    
            \t- Si vous ne le devinez pas au 3è coup, vous perdez votre mise et
            \tvous avez le droit : 
            \t\t- de retenter votre chance avec l'argent qu'il vous reste pour reconquérir le level perdu.
            \t\t- de quitter le jeu.\n
            \t- Dès que vous devinez mon nombre : vous avez le droit de quitter le jeu et de partir avec vos gains OU \n\t\tde continuer le jeu en passant au level supérieur.\n    """)
        while True:   
            if(boolean_question("Retourner au menu (O/N) ?") == True):
                self.wichMode() 
                
    def modeStats(self):
        if(self.userObj.getNiveau_max() == 0):
            print("Aucun stats")
        else:
            ctrUser = UtilisateurController();
            ctrUser.calculStats(self.userObj.getUser_id())
      
        while True:   
            if(boolean_question("Retourner au menu (O/N) ?") == True):
                self.wichMode()
        
    def modeClassement(self):
        entPartie = PartieEntity();
        listClass = entPartie.getClassementGame()
        nbrWin = 0
        print("\n Classement (victoires) :")
        for one_user in listClass:
            print("\t",one_user[1]," : ",one_user[0])
        
        while True:   
            if(boolean_question("Retourner au menu (O/N) ?") == True):
                self.wichMode()
                
        print('oui')
        
            
    def modeQuitter(self):
        print("Merci pour votre visite, à bientot !")
        exit()
    
    def gameCasino(self):
        
        solde = self.userObj.getSolde()
        mise = self.nouvelleMise(solde)
        self.userObj.setSolde(self.updateSolde(solde, mise, 0))
        print("\t- Je viens de penser à un nombre entre 1 et ",tab_level[self.level]['the_range'],". Devinez lequel ?\n")
        print("\t- Att : vous avez le droit à ",tab_level[self.level]['nbr_essaie']," essais !\n")

        nb_python = randrange (1, (10 * self.level))
        print('RESULTAT = ', nb_python )
        nb_coup = 0
        nb_essaie = tab_level[self.level]['nbr_essaie']

        while True :
            if(nb_essaie > nb_coup):
                nb_user = self.rentrerNombre()
                nb_coup += 1
                if nb_user > nb_python :
                    print ('Votre nbre est trop grand')
                elif nb_user < nb_python :
                    print ('Votre nbre est trop petit')
                else :
                    gain = self.calculateGain(nb_coup, mise)
                    solde = self.updateSolde(solde, gain, 1)
                    self.userObj.setSolde(solde)
                    
                    objPartie = Partie(0, mise, self.level, gain, nb_coup, 1, "", self.userObj.getUser_id())
                    entPartie = PartieEntity();
                    entPartie.newPartie(objPartie)
                    self.userObj.setNiveau_max(self.level)
                    print ("Bingo ! Vous avez gagné en {} coup(s) !".format(nb_coup)," et vous avez emporté {} € ! \n".format(gain))
                    return True
            else:
                gain = self.calculateGain(nb_coup, mise)
                solde = self.updateSolde(solde, gain, 0)
                self.userObj.setSolde(solde)
                gain_neg = gain * (-1)
                objPartie = Partie(0, mise, self.level, gain_neg, nb_coup, 0, "", self.userObj.getUser_id())
                entPartie = PartieEntity();
                entPartie.newPartie(objPartie)
                print("Vous avez perdu ! Mon nombre est ",nb_python ," !")
                return False
        
    
    
    def nouvelleMise(self, solde):
        
        mise = input("\t- Le jeu commence, entrez votre mise : ?\n")
        while True:
            try:
                mise = int(mise)
                if mise <= 0 or mise > solde:
                    raise  ValueError()
            except ValueError:
                mise = input("\t- Le montant saisi n'est pas valide. Entrer SVP un montant entre 1 et {} € :  ?\n\t- ".format(solde))
            else:
                return mise
            
    def updateSolde(self, solde, gain, positif):
        
        if(positif == True):
            return solde + gain
        else:
            return solde - gain
        
    def calculateGain(self, nb_coup, mise):
        gain = pay_game[nb_coup] * mise
        return gain
    
    def wichLevel(self):
        reponse = input("- Quel niveau de difficulté souhaitez vous ? (1/2/3) ?")
        while True:
            if reponse.lower() == "1":
                return 1
            elif reponse.lower() == "2":
                return 2
            elif reponse.lower() == "3":
                return 3
            else:
                reponse = input("Réponse incorrect, tapez simplement le niveau souhaité")
            
    def rentrerNombre(self):
        msg = "\t- Entrez SVP votre nombre ? "
        while True:
            try:
                nb_user = int(input(msg))
                break
            except ValueError:
                msg = "\t- Votre réponse n'est pas valide, Entrez SVP votre nombre ? "
                
        return nb_user

    

In [11]:
def boolean_question(question):
    
    restart = input(question)
    while True:
        if restart.lower() == "o":
            return True
        elif restart.lower() == "n":
            return False
        else:
            restart = input("Je ne comprends pas votre réponse, {} ".format(question))

In [ ]:
userCtr = UtilisateurController()
if(boolean_question("Possèdez vous déjà un compte (O/N) ?") == True):
    userObj = userCtr.loginUser()
else:
    userObj = userCtr.registerUser()
partieCtr = PartieController(userObj)
partieCtr.playCore()

In [ ]:
import tkinter as tk

partieEntity = PartieEntity()

regles = [
    "Si vous devinez mon nombre dès le premier coup, vous gagnez le double de votre mise !",
    "Si vous le devinez au 2è coup, vous gagnez exactement votre mise !",
    "Si vous le devinez au 3è coup, vous gagnez la moitiè votre mise !",
    "Si vous ne le devinez pas au 3è coup, vous perdez votre mise et vous avez le droit : ",
    "- de retenter votre chance avec l'argent qu'il vous reste pour reconquérir le level perdu.",
    "- de quitter le jeu.",
    "Dès que vous devinez mon nombre : vous avez le droit de quitter le jeu et de partir avec vos gains OU de continuer le jeu en passant au level supérieur."
]

def afficherMessage(message):
    tk.Label(root, height=1, width=4, text=message, padx="370", pady="10").grid(row=4, column=0)

def voirRegles():
    windowRegles = tk.Toplevel(root)
    windowRegles.title("Règles du Bingo")
    windowRegles.resizable(False, False)
    
    frameRegles = tk.Frame(windowRegles, bd=1)
    frameRegles.grid(row=0, column=0)
    tk.Label(frameRegles, height=1, width=4, text="Règles", padx="320", pady="40",font=("Courier", 44)).grid(row=0, column=0)
    i = 1
    for regle in regles:
        tk.Label(frameRegles, text=regle).grid(row=i, column=0)
        i += 1
        
def creerUtilisateur(pseudo, mdp):
    print(pseudo, mdp)
    
def showIdentifiantsUtilisateur():
    print( e1.get(), e2.get())
    user = verificationIdentifiants(e1.get(), e2.get())
    directionJeu()
    
def showNouveauUtilisateur():
    print(pseudo.get(), mdp.get())
    
def verificationIdentifiants(pseudo, mdp):
    user = Utilisateur(9, "groupe_04", 14.0, "", 3)
    return user

def voirChoixUtilisateur():
    choixUtilisateur = choix.get()
    return choixUtilisateur

def choixUtilisateur():
    choixUtilisateur = voirChoixUtilisateur()
    tk.Label(root, height=1, width=10, padx="300", pady="10", text=choixUtilisateur).grid(row=4, column=0)

def voirClassement():
    windowClass = tk.Toplevel(root)
    windowClass.title("Classement des meilleurs joueurs")
    windowClass.resizable(False, False)
    
    #frameTitle = Frame(windowClass, bd=1)
    #frameTitle.grid(row=0, column=0)
    #Label(frameClass, text="Classement",font=("Courier", 44)).grid(row=0, column=0)
    
    frameClass = tk.Frame(windowClass, bd=1)
    frameClass.grid(row=0, column=0)   
    
    tk.Label(frameClass, text="Mise").grid(row=1, column=0)
    tk.Label(frameClass, text="Level").grid(row=1, column=1)
    tk.Label(frameClass, text="Gain").grid(row=1, column=2)
    tk.Label(frameClass, text="NB coups").grid(row=1, column=3)
    tk.Label(frameClass, text="Résultats").grid(row=1, column=4)
    tk.Label(frameClass, text="Date").grid(row=1, column=5)
    tk.Label(frameClass, text="Pseudo").grid(row=1, column=6)
       
    parties = partieEntity.findAll()
    i = 2
    for partie in parties:
        tk.Label(frameClass, text=partie.getMise()).grid(row=i, column=0)
        tk.Label(frameClass, text=partie.getLevel()).grid(row=i, column=1)
        tk.Label(frameClass, text=partie.getGain()).grid(row=i, column=2)
        tk.Label(frameClass, text=partie.getNbCoup()).grid(row=i, column=3)
        tk.Label(frameClass, text=partie.getResultat()).grid(row=i, column=4)
        tk.Label(frameClass, text=partie.getDateGame()).grid(row=i, column=5)
        tk.Label(frameClass, text=partie.getUser()).grid(row=i, column=5)

def nettoyagePage():
    for widgets in root.winfo_children():
        widgets.destroy()
    
def directionJeu():
    nettoyagePage()
    user = Utilisateur(9, "groupe_04", 14.0, "", 3)
    afficherPageJeu(user)
    
def directionAccueil():
    nettoyagePage()
    afficherPageAccueil()
    
def afficherPageAccueil():
    tk.Label(root, height=1, width=4, text="Bingo", padx="320", pady="40",font=("Courier", 44)).grid(row=0, column=0)    
    tk.Label(root, height=1, width=4, text="Je suis Python. Quel est votre pseudo ?", padx="175", pady="20", font=("Courier", 10)).grid(row=1, column=0)
    tk.Label(root, height=1, width=4, text="Pseudo", padx="370", pady="10").grid(row=2, column=0)
    
    global e1 
    e1 = tk.Entry(root)
    e1.grid(row=3, column=0)

    tk.Label(root, height=1, width=4, text="Mot de passe", padx="370", pady="10").grid(row=4, column=0)
    
    global e2 
    e2 = tk.Entry(root)
    e2.grid(row=5, column=0)

    b1 = tk.Button(root, text="Jouer", command=showIdentifiantsUtilisateur)
    b1.grid(row=6, column=0, pady="10")

    b2 = tk.Button(root, text="Règles", command=voirRegles)
    b2.grid(row=7, column=0, pady="10")

    b3 = tk.Button(root, text="Classement", command=voirClassement)
    b3.grid(row=8, column=0, pady="10")

    tk.Label(root, height=1, width=4, text="Nouveau joueur", padx="175", pady="20", font=("Courier", 20)).grid(row=9, column=0)    
    tk.Label(root, height=1, width=4, text="Pseudo", padx="175", pady="10").grid(row=10, column=0)
    
    pseudo = tk.Entry(root)
    pseudo.grid(row=11, column=0)
    
    tk.Label(root, height=1, width=4, text="Mot de passe", padx="175", pady="10").grid(row=12, column=0)
    
    mdp = tk.Entry(root) 
    mdp.grid(row=13, column=0)
    
    b2 =tk.Button(root, text="Enregistrer").grid(row=14, column=0, padx=5, pady="10")
    
def afficherPageJeu(user):
    tk.Label(root, height=1, width=7, text="Bingo",font=("Courier", 44)).grid(row=0, column=0)
        
    var = "Hello "+user.getPseudo()+" vous avez "+str(user.getSolde())+"€, Très bien ! Installez vous SVP à la table de pari.Je vous expliquerai le principe du jeu :"
    tk.Label(root, height=1, width=10, padx="300", pady="10", text=var).grid(row=1, column=0)
    
    affichage_level = "Level : " + str(level)
    tk.Label(root, height=1, width=10, padx="300", pady="10", text=affichage_level).grid(row=2, column=0)
    
    affichage_nb_coup = "Nombre de coups : " + str(nb_coup)
    tk.Label(root, height=1, width=10, padx="300", pady="10", text=affichage_nb_coup).grid(row=3, column=0)
    
    #tk.Label(root, height=1, width=4, text="Je viens de penser à un nombre entre 1 et 10. Devinez lequel ?", padx="370", pady="10").grid(row=2, column=0)
    afficherMessage("Je viens de penser à un nombre entre 1 et 10. Devinez lequel ?")
    
    global choix
    choix = tk.Entry(root, width=4)
    choix.grid(row=5, column=0, pady="10")
    
    reponse = tk.Button(root, text="Valider", command=choixUtilisateur)
    reponse.grid(row=6, column=0,pady="10")
    
    t1 = tk.Button(root, text="Règles", command=voirRegles)
    t1.grid(row=7, column=0,pady="10")
    
    tk.Button(root, text="Quitter", command=directionAccueil).grid(row=8, column=0)
    
    

root = tk.Tk()
root.title("Casino")
root.resizable(False, False)
root.geometry("768x768")

level = 1

nb_coup = 0
afficherPageAccueil()

root.mainloop()